In [1]:
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
import gradio as gr
from langchain_groq import ChatGroq
import streamlit as st
import os
from groq import Groq
import bs4

c:\RetrievalAugumentedGeneration_DB_website\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
load_dotenv() 

# os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
groq_api_key = os.environ['GROQ_API_KEY']


In [2]:
userInput=st.text_input("Enter website links")
[userInput]

2024-04-07 21:29:48.448 
  command:

    streamlit run c:\RetrievalAugumentedGeneration_DB_website\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


['']

In [3]:
urls=[
    "https://kamalesh.net/",
    "https://www.flipkart.com/motorola-edge-50-pro-5g-125w-charger-luxe-lavender-256-gb/p/itm3281f9ac32d1a?pid=MOBGXFXY2SBHGSVN&lid=LSTMOBGXFXY2SBHGSVNUNNNTJ&marketplace=FLIPKART&q=moto+edge+50+pro&store=tyy%2F4io&srno=s_1_1&otracker=AS_QueryStore_OrganicAutoSuggest_2_9_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_9_na_na_na&fm=organic&iid=f70e0ce2-f55d-4e98-b74b-96514017f77a.MOBGXFXY2SBHGSVN.SEARCH&ppt=hp&ppn=homepage&ssid=jst76zkm7k0000001712501090760&qH=22f8a288f3cd4f21"
    
]

text_documents=[WebBaseLoader(url).load() for url in urls]


In [4]:
text_documents

[[Document(page_content="\n\n\n\n\nKamalesh\n\n\n\n\n\n\n\n\n\nHome\n\n\n\nSkills\n\nEmbedded Sytems\nBusiness Analytics\nProduct Design & SQA\nProgramming\n\n\n\nLinks\n\n\n\n\n\nHello, I'm Kamalesh\nMultidisciplinary Engineer with good exposure to Embedded systems, Programming, Mechanical and Business Analytics. \n\nAbout Me\n\n\n\n\n\n\nAbout Me\n\n\n\n\n\n\nI am a passion driven, result oriented Multidisciplinary Engineer with high proficiency in\r\n        Mechanical, Electrical and Computer Science Engineering.\nI set goals for myself and strive to achieve it through consistent learning and improvement cycle.\r\n        Started my career as a Mechanical Engineer, developed Product Design and People management skillsets.\r\n         Continued to develop my skills on Embedded systems, Mechatronics System integration, Programming and Business Analytics.\r\n        \n\n\n\n\n\n\n\nEmbedded System\n\n\n\n\nAdequate first-hand experience on using multitude of Microcontrollers.\nWorked 

In [6]:
docs_list=[item for sublist in text_documents for item in sublist]
docs_list

[Document(page_content="\n\n\n\n\nKamalesh\n\n\n\n\n\n\n\n\n\nHome\n\n\n\nSkills\n\nEmbedded Sytems\nBusiness Analytics\nProduct Design & SQA\nProgramming\n\n\n\nLinks\n\n\n\n\n\nHello, I'm Kamalesh\nMultidisciplinary Engineer with good exposure to Embedded systems, Programming, Mechanical and Business Analytics. \n\nAbout Me\n\n\n\n\n\n\nAbout Me\n\n\n\n\n\n\nI am a passion driven, result oriented Multidisciplinary Engineer with high proficiency in\r\n        Mechanical, Electrical and Computer Science Engineering.\nI set goals for myself and strive to achieve it through consistent learning and improvement cycle.\r\n        Started my career as a Mechanical Engineer, developed Product Design and People management skillsets.\r\n         Continued to develop my skills on Embedded systems, Mechatronics System integration, Programming and Business Analytics.\r\n        \n\n\n\n\n\n\n\nEmbedded System\n\n\n\n\nAdequate first-hand experience on using multitude of Microcontrollers.\nWorked w

In [12]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs_list)

In [13]:
documents

[Document(page_content="Kamalesh\n\n\n\n\n\n\n\n\n\nHome\n\n\n\nSkills\n\nEmbedded Sytems\nBusiness Analytics\nProduct Design & SQA\nProgramming\n\n\n\nLinks\n\n\n\n\n\nHello, I'm Kamalesh\nMultidisciplinary Engineer with good exposure to Embedded systems, Programming, Mechanical and Business Analytics. \n\nAbout Me\n\n\n\n\n\n\nAbout Me\n\n\n\n\n\n\nI am a passion driven, result oriented Multidisciplinary Engineer with high proficiency in\r\n        Mechanical, Electrical and Computer Science Engineering.\nI set goals for myself and strive to achieve it through consistent learning and improvement cycle.\r\n        Started my career as a Mechanical Engineer, developed Product Design and People management skillsets.\r\n         Continued to develop my skills on Embedded systems, Mechatronics System integration, Programming and Business Analytics.\r\n        \n\n\n\n\n\n\n\nEmbedded System", metadata={'source': 'https://kamalesh.net/', 'title': 'Kamalesh', 'language': 'en'}),
 Document(p

In [14]:
db=Chroma.from_documents(
    documents=documents,
    collection_name="rag-chroma",
    embedding=OllamaEmbeddings(model='nomic-embed-text'),
    
)

In [18]:
query="What are skills of Kamalesh?"

result=db.similarity_search(query)
result[0].page_content

"Kamalesh\n\n\n\n\n\n\n\n\n\nHome\n\n\n\nSkills\n\nEmbedded Sytems\nBusiness Analytics\nProduct Design & SQA\nProgramming\n\n\n\nLinks\n\n\n\n\n\nHello, I'm Kamalesh\nMultidisciplinary Engineer with good exposure to Embedded systems, Programming, Mechanical and Business Analytics. \n\nAbout Me\n\n\n\n\n\n\nAbout Me\n\n\n\n\n\n\nI am a passion driven, result oriented Multidisciplinary Engineer with high proficiency in\r\n        Mechanical, Electrical and Computer Science Engineering.\nI set goals for myself and strive to achieve it through consistent learning and improvement cycle.\r\n        Started my career as a Mechanical Engineer, developed Product Design and People management skillsets.\r\n         Continued to develop my skills on Embedded systems, Mechatronics System integration, Programming and Business Analytics.\r\n        \n\n\n\n\n\n\n\nEmbedded System"

In [19]:
query="What is the Specification of Moto Edge 50 Pro"

result=db.similarity_search(query)
result[0].page_content

'MOTOROLA Edge 50 Pro 5G ( 256 GB Storage, 12 GB RAM ) Online at Best Price On Flipkart.com       Explore PlusLoginBecome a Seller More CartBig Bachat Days Ends in03 hrs : 44 mins : 10 secsHomeMobiles & AccessoriesMobilesMOTOROLA MobilesMotorola Edge 50 Pro 5G with 125W Charger (Luxe Lavender, 256 GB) (12 GB RAM)'

In [20]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000248E18F6450>)

In [28]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [29]:

chat = ChatGroq(
    api_key = groq_api_key,
    model_name = "mixtral-8x7b-32768"
)

In [30]:
document_chain = create_stuff_documents_chain(chat, prompt)


In [33]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [34]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000248E18F6450>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='Answer the following question based only on the provided context:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}'))])
            | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000248E0185590>, async_client=<groq.resources.chat.completions.AsyncC

In [35]:

response = retrieval_chain.invoke({"input": "What are the skills of Kamalesh?"})
print(response["answer"])

Based on the provided context, Kamalesh's skills are:

1. Embedded Systems: Kamalesh has experience with Embedded Systems and Mechatronics System integration.
2. Business Analytics: Kamalesh has knowledge of Business Analytics.
3. Product Design & SQA: Kamalesh has experience as an R&D and SQA Engineer, strong command over Machine Design and Strength of Machines, and proficiency in softwares like Creo, Solidworks, ANSYS, and Matlab/Simulink.
4. Programming: Kamalesh is adept at programming in C, has exposure to Image Processing and libraries like OpenCV, MediaPipe, and NumPy in Python, knowledge of Data Analytics using languages like R and Python and libraries like pandas, seaborn, matplotlib, plotly for Data Visualization, and hands-on exposure in using Version control using Git.
5. Mechanical Engineering: Kamalesh has a strong background in Mechanical Engineering.
6. Electrical and Computer Science Engineering: Kamalesh has high proficiency in Electrical and Computer Science Engineer

In [ ]:
st.()

In [36]:

response = retrieval_chain.invoke({"input": "What is the Spec of Moto edge 50 pro?"})
print(response["answer"])

Based on the provided context, here are some of the specifications of the Motorola Edge 50 Pro:

* Display: 6.7-inch 1.5K super HD pOLED display, HDR10+, 10-bit, 144Hz refresh rate, 360Hz touch rate (gaming mode), 2000 nits peak brightness, Pantone validated for true colors, Screen-to-body ratio of 90.75%
* Resolution: 2712 x 1220 pixels
* Processor: Snapdragon 7 Gen 3 Mobile Platform with Octa Core processor
* Operating System: Android 14
* Storage: 256 GB
* RAM: 12 GB
* Rear Camera: 50MP with f/1.4 aperture, Quad Pixel Technology for 1.28um, Optical Image Stabilization (OIS), Super Resolution Zoom, Hybrid Zoom: 30X, Burst Shot, Timer, Assistive Grid, Leveler, Metering Mode, Watermark, Live Filters, Selfie Photo Mirror, Selfie Stick Support, RAW Photo Output, QR/Barcode Scanner, Quick Capture (Twist-Twist), 13 MP ultrawide + macro camera, 10 MP telephoto camera with OIS
* Front Camera: 50MP with f/1.9 aperture, 0.64um Pixel Size, Quad Pixel Technology for 1.28um, Autofocus, Features: 

In [ ]:
iface = gr.Interface(
    fn = fetch_response,
    inputs = "text",
    outputs = "text",
    title = "Groq Chatbot",
    description="Ask a question and get a response."
)

iface.launch()

In [4]:
st.Textbox

False